In [1]:
import pandas as pd
import numpy as np
import re
import unidecode
import demoji
import nltk
from nltk.corpus import stopwords
from spacy.lang.pt.stop_words import STOP_WORDS
import spacy
from string import punctuation
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import tokenize
import nltk

import gensim
import sys
from gensim.models.phrases import Phraser
import gensim.corpora as corpora
from pprint import pprint

from tqdm import tqdm
import tqdm
from gensim.models import CoherenceModel
%matplotlib inline

In [2]:
nlp = spacy.load("pt")
nlp.Defaults.stop_words |= {"gt", "&gt", "to","uol","mi","budddhetg","the", "ne", "vou", "ta", "via","ex", "pq", "vc","aa","pra","to", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"}
palavras_irrelevantes = set(stopwords.words('portuguese') + list(punctuation))

In [3]:
def cleanText(tweet):
    if "|" in tweet:
        tweet = tweet.split("|")[1]
    
    tweet = ' '.join(re.sub(r"http\S+", "", tweet).split())
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([0-9])", " ", tweet).split())
    tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
    tweet = ' '.join(re.sub("[\_\|\.\,\"\'\!\?\:\;\$\-\(\)\=]", " ", tweet).split())
    tweet = tweet.lower()
    
    le = list(demoji.findall(tweet))
    for i in le:
        tweet = tweet.replace(i, "")
    
    if tweet.startswith('rt '):
        tweet = tweet.replace("rt ", "")
    
    lNewTweet = []
    for i in tweet.split(" "):
        if i not in palavras_irrelevantes and i not in STOP_WORDS:
            lNewTweet.append(i)
    
    newTweet = " ".join(lNewTweet)
        
    return unidecode.unidecode(newTweet.replace(" rt ", ""))

In [4]:
data = pd.read_csv("data/dataTweeter.csv", sep=";", encoding='utf8')
data.head()

,ID,USER,POST
0,1376863023600599043,g1,Veja a previsão de entrega de vacinas da Fiocr...
1,1376859250056519687,g1,Presidente trocou comandos de Relações Exterio...
2,1376855473828290562,g1,Vítima era obrigada a realizar serviços domést...
3,1376851704335978500,g1,Os dois defendiam a vacinação nas redes sociai...
4,1376847923758108672,g1,Leia os bastidores no blog do @gcamarotti http...


In [5]:
data["tweet_text_clean"] = data.POST.apply(cleanText)
data.drop(columns=["ID", "USER", "POST"], inplace=True)
data.head()

,tweet_text_clean
0,previsao entrega vacinas fiocruz butantan
1,presidente trocou comandos relacoes exteriores...
2,vitima realizar servicos domesticos remuneraca...
3,defendiam vacinacao redes sociais
4,leia bastidores blog


In [6]:
token_ = [doc.split(" ") for doc in data["tweet_text_clean"].tolist()]

In [7]:
bigram = gensim.models.Phrases(token_, min_count=1, threshold=0.1) # higher threshold fewer phrases
trigram = gensim.models.Phrases(bigram[token_], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

l = [trigram_mod[bigram_mod[doc]] for doc in token_]
# print(l)

In [8]:
token_3 = []
token_2 = [[token_3.append(y) for y in x] for x in token_]

frequencia = nltk.FreqDist(token_3)
freqmc = frequencia.most_common(10000)
# freqmc

In [9]:
id2word = corpora.Dictionary(l)
corpus = [id2word.doc2bow(text) for text in l]

In [10]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.31,
                                           eta=0.01)

In [11]:
# Print the Keyword in the 10 topics
# doc_lda = lda_model[corpus]
# print(dir(doc_lda))
# print(doc_lda.corpus)
# pprint(lda_model.print_topics())
# pprint(dir(lda_model))
# pprint(lda_model.get_topics())
# print(lda_model.show_topic(1))
lt = []
for i, row in enumerate(lda_model[corpus]):
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    lt.append(row[0][0])

# print(lt)
contents = pd.Series(lt, name="topic")
data_new = pd.concat([data, contents], axis=1)

In [12]:
data_new[data_new["topic"] == 2].sample(10)["tweet_text_clean"].tolist()

['situacao brasil alerta manter virus controle requer atencao continua autoridades lideres saude publica',
 'edicao participantes capricharam vocabulario conseguiu acompanhar brigas discussoes casa direitinho',
 'brasileira desenhos cenarios filme mank concorre oscar confira trabalho',
 'tv globo conasems disse ocorreu falha comunicacao pediu retirada campos obrigatorios',
 'ultima semana falamos linha frente vacinacao troca ministro saude cerco criticos bolsonaro juros basicos alta esgotamentos utis perdeu algum episodio ouca',
 'ultima semana falamos linha frente vacinacao troca ministro saude cerco criticos bolsonaro juros basicos alta esgotamentos utis perdeu algum episodio ouca',
 'taos melhor compass lider mercado',
 'ministerio saude abre consulta publica inclusao derivado cannabis sus',
 'video gravado casa vizinha governador sp entenda',
 'assessoria cantor apresenta avancos significativos recuperacao conversado diariamente irmao telefone']